In [1]:
import random
random.seed(42)  

In [2]:
import math
import cv2
import numpy as np
import os 
import glob 
from sklearn.cluster import KMeans
from sklearn import svm
from functools import reduce
import operator
from sklearn import metrics
from IPython.core.display import Image, display
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [3]:
# defining feature extractor that we want to use
extractor = cv2.xfeatures2d.SIFT_create()
def features(image, extractor):
    keypoints, descriptors = extractor.detectAndCompute(image, None)
    return keypoints, descriptors

In [4]:
def build_histogram(descriptor_list, cluster_alg):
    
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return  histogram / len(cluster_result)

In [ ]:
def onedossier(i):
 file_dir = 'C:\\Users\\LENOVO\\damage//*'
 files = glob.glob(file_dir)
 descriptor_list=np.empty(shape=[1, 128])
 x=[]
 descriptor=[] 
 img_dir = os.path.join(files[i],'*')   
 images = glob.glob(img_dir)
 if (len(images)==0):
    print('empty folder')    
 else:   
    for img in images:
        img_temp = cv2.imread(img)
        img_temp=cv2.cvtColor(img_temp, cv2.COLOR_BGR2GRAY)
        img_temp = cv2.resize(img_temp,(512,512))
        x.append(img_temp)
        keypoint,descriptors = features(img_temp, extractor)
        if(descriptors is not None):
         descriptors[~np.isnan(descriptors).any(axis=1)]  
         descriptor_list=np.concatenate((descriptor_list, descriptors),axis=0) 
                
 return  descriptor_list,x

In [ ]:
file_dir = 'C:\\Users\\LENOVO\\damageprim//*'
files = glob.glob(file_dir)

In [ ]:
def validation(i,kmeans):
 val_image2=[]
 #random.seed(4)  
 for q in range(len(files)):
  if (q!=i)  :
   img_dir = os.path.join(files[q],'*')
   images = glob.glob(img_dir)
   sampling = random.choices(images, k=1)
   print('this is sampling:',sampling)
   data = cv2.imread(sampling[0])
   data =cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
   data =cv2.resize(data,(512,512))
   keypoint, descriptors = features(data, extractor)  
   descriptors[~np.isnan(descriptors).any(axis=1)]
   histogram = build_histogram(descriptors, kmeans)
   val_image2.append(histogram)  
 return val_image2 

In [ ]:
index=[x for x in range(15)]
random.shuffle(index)
num1=index[:11]
print('this is num1:',num1)
num2=index[11:]
print('this is num2:',num2)


this is num1: [8, 13, 7, 6, 14, 12, 5, 2, 9, 3, 4]
this is num2: [11, 0, 1, 10]


In [ ]:
def optimize_training_parameters(i):
        scoreinit=0
        opt_leaf=3
        opt_depth=3
        opt_estimator =30
        opt_base_estimator=3
        opt_nbclusters=50
        random.seed(4)  
        for k in [100,129,150,200,241,250,300]:
        
         if (scoreinit<1):
          kmeans = KMeans(n_clusters =k,random_state=5)
          print('opt',i)
          [l1,l2]=onedossier(i)
        
          l1=np.nan_to_num(l1)
          l1[~np.isnan(l1).any(axis=1)]
          kmeans.fit(l1) 
          preprocessed_image=[]  
          for f in (l2): 
                 
                 keypoint,descriptors = features(f, extractor)  
                 if(descriptors is not None):
                     descriptors[~np.isnan(descriptors).any(axis=1)]
                     histogram = build_histogram(descriptors, kmeans)
                     preprocessed_image.append(histogram)
                     
        
         
        
         
          preprocessed_image1=[preprocessed_image[b] for b in num1]
          val_image1=[preprocessed_image[b] for b in num2]
          target1=np.ones(len(val_image1)) 
          out=validation(i,kmeans)
          val_image2=out[:4]
          target2=-1*np.ones(len(val_image2))
          val_image=val_image1+val_image2
          target=np.concatenate((target1,target2))
          y_train=np.ones(11)
          random.seed(4)  
          for n_estimator in [40, 42,70,100,300] :
           if (scoreinit<1):
            for samples in [6,10,11,5]:
                    if (scoreinit<1):
                         for leaf  in[3,5,10,15] :
                          if (scoreinit<1):     
                           for depth in [3,5,8,10,15] :
                            if (scoreinit<1):
                             dt = DecisionTreeClassifier(max_leaf_nodes=leaf, max_depth=depth)
                             bc = BaggingClassifier(base_estimator=dt,n_estimators=n_estimator ,max_samples=samples, oob_score=True, random_state=1) #n_estimators=70, random_state=1)
                             bc.fit(preprocessed_image1,y_train)
                             preds = bc.predict(val_image)
                             print(type(preds))
                             accuracy=metrics.f1_score(target, preds) 
                            
                             y_pred = bc.predict(preprocessed_image1)
                             
                             print(y_pred)
                             print(preds)
                             print(target)   
                             print('this is acc',accuracy)  
                             print('this is sc',scoreinit) 
                             if (scoreinit < accuracy):
                            
                                        scoreinit=accuracy
                                        trainig_img=preprocessed_image1
                                        opt_estimator = n_estimator 
                                        opt_samples=samples
                                        opt_nbclusters=k
                                        opt_leaf=leaf
                                        opt_depth=depth
                                        
                                        if (scoreinit==1):
                                           break
        print('out of loop')                        
                                   
        return  opt_estimator ,opt_samples ,opt_nbclusters,opt_leaf,opt_depth,trainig_img

In [ ]:
list4=[]
listopt=[]
for z in range(1):
     print(z) 
     #random.seed(42)     
     M=optimize_training_parameters(z)
     listopt.append(M)   
     print('finish')
     kmeans = KMeans(n_clusters =listopt[z][2],random_state=42)
     [l1,l2]=onedossier(z) 
     l1=np.nan_to_num(l1)   
     l1[~np.isnan(l1).any(axis=1)]  
     kmeans.fit(l1)
     list4.append(kmeans)   

0
opt 0
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage10\\13.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage2\\10.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage3\\14.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage4\\11.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage5\\1.JPG']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage6\\14.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage7\\8.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage8\\6.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damageprim\\damage9\\5.jpg']
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

...

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1. -1. -1. -1. -1.]
this is acc 0.6666666666666666
this is sc 0.6666666666666666
<class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  1.  1.  1

In [ ]:
print(listopt)
print(num1)
print(num2)
print(list4)

In [ ]:
file_dir ='C:\\Users\\LENOVO\\nouveau images1/*'
file = glob.glob(file_dir)
print(file)
print(len(file))

In [ ]:
myorder = [0, 1, 3, 4, 5,6,7,9,10,12,2,8,11,13]
print(len(file))
file = [file[h] for h in myorder]
print(file)

In [ ]:
def test(g):
 test_image=[]
 for f1 in file:
  data = cv2.imread(f1)
  data =cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
  data =cv2.resize(data,(512,512))
  keypoint, descriptors = features(data, extractor)  
  descriptors[~np.isnan(descriptors).any(axis=1)]  
  histogram = build_histogram(descriptors, list4[g])
  test_image.append(histogram) 
 return(test_image)  

In [ ]:
list2=[]
list3=[]
#for i in range(len(files)):
for i in range(1):    
  
  if((listopt[i][0]!=None)&(listopt[i][1]!=None)&(listopt[i][2]!=None)&(listopt[i][3]!=None)&(listopt[i][4]!=None)&(listopt[i][5]!=None)): 
     dt = DecisionTreeClassifier(max_leaf_nodes=listopt[i][3], max_depth=listopt[i][4])
     clf=BaggingClassifier(base_estimator=dt,n_estimators=listopt[i][0] ,max_samples=listopt[i][1], oob_score=True, random_state=1)
     y_train=np.ones(11)  
     clf.fit(listopt[i][5],y_train)
     y_pred_train = clf.predict((listopt[i][5]))
     print('avant')
     y_pred_test = clf.predict(test(i))
     print('apres')
  list3.append(y_pred_train)
  list2.append(y_pred_test)
print('khouloud')

In [ ]:
print(list3)
print(list2)